# 路由

Flask提供了丰富的路由绑定方式,最适合Rest风格api服务的绑定方式是`基于方法调度`的绑定方式,它需要使用flask的`flask.views.MethodView`作为基类并使用app的`add_url_rule`方法来与url绑定

其中,各个类的对应http方法的方法所定义的过程就是客户端用http对应方法访问时的处理过程

> 例: 一个简单的helloworld服务

In [19]:
%%writefile codes/c1/app.py
# --*-- coding:utf-8 --*--
from __future__ import print_function, unicode_literals, division

from flask import Flask,jsonify
from flask.views import MethodView

app = Flask(__name__)

class HelloAPI(MethodView):
    
    def get(self):
        return jsonify({"result":"hello!"})
        

app.add_url_rule('/hello/',view_func=HelloAPI.as_view('hello'.encode("utf-8")))

if __name__ == '__main__':
    app.run(debug=True)

Overwriting codes/c1/app.py


这个程序的基本结构是

1. 定义编写的编码和使用的特性
2. 引入需要的依赖
3. 定义app实例
4. 定义view
5. 绑定路由
6. 定义如果是直接运行脚本,则使用测试服务器运行这

我们可以在对应目录下运行这个脚本试试.默认端口是5000

In [4]:
import requests

In [28]:
requests.get("http://127.0.0.1:5000/hello/").json()

{u'result': u'hello!'}

## rest API和复杂的路由

Web API 的工作通常与 HTTP 动词方法紧密相关，所以这使得实现这样一个基于 MethodView 类的 API 很有意义。也就是说，你会注意到大多数时候， API 需要不同的 URL 规则来访问相同的方法视图。譬如，想象一种情况，你在 web 上暴露一个用户对象:


URL	|HTTP方法	|描述
---|---|---
/users/|	GET|	获得全部用户的列表
/users/	|POST	|创建一个新用户
`/user/<id>`	|GET	|显示某个用户
`/user/<id>`	|PUT	|更新某个用户
`/user/<id>`	|DELETE	|删除某个用户

让我们来实现这个视图的路由,我们先用字典描述数据

In [47]:
%%writefile codes/c1/app2.py
# --*-- coding:utf-8 --*--
from __future__ import print_function, unicode_literals, division

from flask import Flask,jsonify,request
from flask.views import MethodView
import json
app = Flask(__name__)

Userlist = []
def inc(cls):
    counter = {"result":0}
    def wrap(*args):
        counter["result"]+=1
        result = cls(counter["result"],*args)
        global Userlist
        Userlist.append(result)
        return result
    return wrap
@inc
class User(object):
    def __str__(self):
        return "<USER: id-{self._id}-{self.name}>".format(self=self)
    def __repr__(self):
        return self.__str__()
    def __init__(self,_id,name,password,type="local"):
        self._id=_id
        self.name = name
        self.password = password
        self.type=type
        
class UsersAPI(MethodView):
    
    def get(self):
        return jsonify({"result":map(lambda x:x.__str__(),Userlist)})
        
    def post(self):
        data = json.loads(request.data)
        user = User(data.get("name"),data.get("password"))
        return jsonify({"result":"save {name} done!".format(name=user.name)})
    
class UserAPI(MethodView):
    
    def get(self,user_id):
        user = filter(lambda x:True if x._id==user_id else False,Userlist)[0]
        result = {"name":user.name,"password":user.password,"type":user.type}
        return jsonify({"result":result})
        
    def put(self,user_id):
        data = request.args
        user = filter(lambda x: True if x._id==user_id else False,Userlist)[0]
        if request.args.get("name"):
            user.name=data.get("name")
        if request.args.get("password"):
            user.password=data.get("password")
        
        return jsonify({"result":"update {name} done!".format(name=user_id)})    
    def delete(self,user_id):
        global Userlist
        Userlist = filter(lambda x: False if x._id==user_id else True,Userlist)
        return jsonify({"result":"delete {id} done!".format(id=user_id)})   
        

app.add_url_rule('/users/',view_func=UsersAPI.as_view('users'.encode("utf-8")))
app.add_url_rule('/user/<int:user_id>',view_func=UserAPI.as_view('user'.encode("utf-8")))

if __name__ == '__main__':
    app.run(debug=True)

Overwriting codes/c1/app2.py


In [48]:
import json

In [49]:
requests.post("http://127.0.0.1:5000/users/",data = json.dumps({"name":"niuniu","password":"qwe"}),
              headers={'content-type': 'application/json'}).json()

{u'result': u'save niuniu done!'}

In [50]:
requests.get("http://127.0.0.1:5000/users/").json()

{u'result': [u'<USER: id-1-niuniu>']}

In [51]:
requests.get("http://127.0.0.1:5000/user/1").json()

{u'result': {u'name': u'niuniu', u'password': u'qwe', u'type': u'local'}}

In [52]:
requests.put("http://127.0.0.1:5000/user/1",params={"password":"1234"}).json()

{u'result': u'update 1 done!'}

In [53]:
requests.get("http://127.0.0.1:5000/user/1").json()

{u'result': {u'name': u'niuniu', u'password': u'1234', u'type': u'local'}}

In [54]:
requests.delete("http://127.0.0.1:5000/user/1").json()

{u'result': u'delete 1 done!'}

In [55]:
requests.get("http://127.0.0.1:5000/users/").json()

{u'result': []}